In [1]:
import numpy as np
from sklearn.linear_model import LogisticRegression
import pandas as pd
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

In [2]:
data = pd.read_csv('../openreview.csv')
binary_decisions = {'Accept (Oral)':1, 'Accept (Poster)':1, 'Accept (Spotlight)':1, 'Accept (Talk)':1, 'Withdrawn':0, 'Reject':0, 'Invite to Workshop Track':0}

In [3]:
def logistic_regression():
    scores =[]
    theorem_indicator=[]
    target=[]

    for rating, decision, theorem, in zip(data['ratings'], data['decisions'], data['theorem']):
        if theorem == 'pdf miss':
            continue
        if theorem =='y':
            theorem_indicator.append(1)
        else:
            theorem_indicator.append(0)

        avg_rating=0
        for i in rating.split(';'):
            avg_rating += int(i)
        scores.append(avg_rating/len(rating.split(';')))

        target.append(binary_decisions[decision])

    x=pd.DataFrame()
    x['mean reviewer score']=scores
    x['theorem']=theorem_indicator
    x['constant']=[1]*len(scores)
    y=np.array(target)

    logit_model = sm.Logit(y, x)
    summary = logit_model.fit().summary()

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.3, random_state=0)
    logisticRegr = LogisticRegression(random_state=0)
    model = logisticRegr.fit(x_train, y_train)
    accuracy = logisticRegr.score(x_test, y_test)

    return summary, accuracy

In [6]:
summary, score = logistic_regression()
print(summary)
print('Accuracy: ', score)

Optimization terminated successfully.
         Current function value: 0.245878
         Iterations 9
No theorem:  [[0.94415939 0.05584061]]
Has theorem:  [[0.93987911 0.06012089]]
Optimization terminated successfully.
         Current function value: 0.610953
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                 8514
Model:                          Logit   Df Residuals:                     8511
Method:                           MLE   Df Model:                            2
Date:                Thu, 29 Jul 2021   Pseudo R-squ.:                  0.5993
Time:                        14:25:00   Log-Likelihood:                -2093.4
converged:                       True   LL-Null:                       -5224.2
Covariance Type:            nonrobust   LLR p-value:                     0.000
                          coef    std err          z      P>|z|      [0.025      0.97